# Refactored SigBERT with CSAThis notebook demonstrates the modularized pipeline for Survival Analysis and Conformal Prediction.

In [ ]:
%load_ext autoreload%autoreload 2import sysfrom pathlib import Pathimport pandas as pdimport numpy as np# Add src to path if running from notebooks dirif Path.cwd().name == "notebooks":    sys.path.append(str(Path.cwd().parent))    from src.config import SEED, setup_plotting_stylefrom src.data_loader import load_df, find_data_dirfrom src.preprocessing import split_dataset, process_features, profile_datasetfrom src.models import train_cox_cv, train_rsf_cv, predict_surv_on_grid_cox, predict_surv_on_grid_rsf, bounded_median_timefrom src.csa import CSAPredictorfrom src.evaluation import compute_performance_metricsfrom src.visualization import plot_curves, plot_csa_width_hist, plot_individual_predictionsetup_plotting_style()

In [ ]:
# 1. Load Data# Ensure your CSV files are in the data/ directoryDATA_DIR = find_data_dir()print(f"Loading data from: {DATA_DIR}")try:    df_main = load_df(DATA_DIR / "df_study_L36_w6.csv")    print(f"Data loaded: {df_main.shape}")except FileNotFoundError:    print("Dataset not found. Please place 'df_study_L36_w6.csv' in the 'data' folder.")

In [ ]:
# 2. Preprocessing# Split: Train (50%), Stratifieddf_train, df_calib, df_test = split_dataset(df_main)print(f"Train: {df_train.shape[0]}, Calib: {df_calib.shape[0]}, Test: {df_test.shape[0]}")# Profileprint(profile_dataset(df_train, "Train"))# Feature Processing (Impute, Select, Scale)# Signal columns assumed to start with 'sig_'from src.data_loader import get_sig_nonconstsig_cols, _ = get_sig_nonconst(df_train)X_train, X_calib, X_test, selected_features = process_features(df_train, df_calib, df_test, sig_cols)print(f"Selected features: {len(selected_features)}")

In [ ]:
# 3. Model Training# Cox ElasticNetcox_results = train_cox_cv(X_train, df_train)cox_model = cox_results["model"]print(f"Best Cox Params: {cox_results['best_params']}")# RSFrsf_results = train_rsf_cv(X_train, df_train)rsf_model = rsf_results["model"]print(f"Best RSF Score: {rsf_results['best_params']}")

In [ ]:
# 4. Evaluation# Define Gridt_min = df_train["time"].min()t_max = df_train["time"].max()grid = np.linspace(t_min, t_max, 200)# Predictions on gridsurv_cox = predict_surv_on_grid_cox(cox_model, X_test, grid)surv_rsf = predict_surv_on_grid_rsf(rsf_model, X_test, grid)# Risk scores (higher = riskier)risk_cox = cox_model.predict_partial_hazard(X_test).values.ravel()risk_rsf = rsf_model.predict(X_test)# Metricsperf_cox = compute_performance_metrics(risk_cox, surv_cox, df_train, df_test, grid)perf_rsf = compute_performance_metrics(risk_rsf, surv_rsf, df_train, df_test, grid)print("Cox Performance:", {k: v for k, v in perf_cox.items() if "times" not in k})print("RSF Performance:", {k: v for k, v in perf_rsf.items() if "times" not in k})# Visualizationif perf_cox['auc_scores'] is not None:    plot_curves(perf_cox['auc_times'], {"Cox": perf_cox['auc_scores'], "RSF": perf_rsf['auc_scores']},                "Dynamic AUC", "AUC(t)")

In [ ]:
# 5. Conformal Survival Analysis (CSA)# Predict median times for Calibration# Pre-compute survival on grid for Calib (Cox model used as base)surv_cox_calib = predict_surv_on_grid_cox(cox_model, X_calib, grid)mu_calib = np.array([bounded_median_time(row, grid) for row in surv_cox_calib])# Predict median times for Testsurv_cox_test = surv_cox # Already computedmu_test = np.array([bounded_median_time(row, grid) for row in surv_cox_test])# Fit CSAcsa = CSAPredictor(alpha=0.10)csa.fit(df_train, df_calib, mu_calib, grid)# Predict Intervalslower, upper = csa.predict(mu_test, grid)# Evaluatestats = csa.evaluate(df_test, lower, upper)print("CSA Stats:", stats)# Plotplot_csa_width_hist(upper - lower)# Individual Exampleidx = 0plot_individual_prediction(grid, surv_cox_test[idx], lower[idx], upper[idx],                           df_test.iloc[idx]["time"], df_test.iloc[idx]["event"],                           f"Patient {idx}")